In [1]:
import rasterio as rio
import numpy as np
from PIL import Image
def load_data(path_dict):
    data_dict = dict()
    for key, value in path_dict.items():
        with rio.open(value) as src:
            data = src.read()
            data = np.squeeze(data).astype("float32")
            data_dict[key] = (data, src)
    return data_dict
def get_lat_long(data, src):
    # index array
    lat_index = np.arange(0, data.shape[0])
    long_index = np.arange(0, data.shape[1])
    
    # meshgrid
    long_grid, lat_grid = np.meshgrid(long_index, lat_index)
    
    # flattened grids
    lat_grid_flat = lat_grid.flatten()
    long_grid_flat = long_grid.flatten()
    
    # getting long and lat
    A = src.transform
    long, lat = rio.transform.xy(A, lat_grid_flat, long_grid_flat)
    
    # reshaping to shape of original data
    lat = np.array(lat).reshape(data.shape)
    long = np.array(long).reshape(data.shape)
    
    return lat, long

def segment_coords(data, lat, long, target_coords = (0, 0), border = 0, length = 10):
    # latitude/longitude variables
    lat_max = target_coords[0]
    lat_min = lat_max - length
    long_min = target_coords[1]
    long_max = long_min + length
    
    # length variables
    xlen = data.shape[1]
    ylen = data.shape[0]
    
    # find xmin and xmax
    xmin, xmax = None, None
    for x in range(xlen):
        if long[0, x] > long_min:
            xmin = x if xmin == None else xmin
        if long[0, xlen - x - 1] < long_max:
            xmax = xlen - x - 1 if xmax == None else xmax
            
    # find ymin and ymax
    ymin, ymax = None, None
    for y in range(ylen):
        if lat[ylen - y - 1, 0] > lat_min:
            ymin = ylen - y - 1 if ymin == None else ymin
        if lat[y, 0] < lat_max:
            ymax = y if ymax == None else ymax
            
    # index data from top-bottom, left-right
    return data[ymax:ymin + border, xmin:xmax + border]

In [ ]:
years = (2000,2005, 2010, 2015)
path_dict = {}
for yr in years:
    pop_data_path = "../data/raw/population/gpw_v4_population_count_rev11_{yr}_30_sec.tif".format(yr = yr)
    path_dict["pop_{yr}".format(yr = yr)] = pop_data_path

# Segmenting Maize Data 10S60W
target_coords = (-10, -60)
data_dict = load_data(path_dict)
lat, long = get_lat_long(data_dict["pop_2000"][0], data_dict["pop_2000"][1])
for year in years:
    soy_seg = segment_coords(data_dict["pop"][0], lat, long, target_coords, border = 1, length = 10)
    soy_seg = Image.fromarray(soy_seg)
    soy_seg.save("../data/raw/segmented/{lat}{long}/pop_{yr}_{lat}{long}.tif".format(lat = "10S", long = "60W", yr = year))